In [ ]:
#Import packages and define functions
import json
import os
from PIL import Image
from ultralytics import YOLO
import shutil
import random
import torch

if torch.cuda.is_available():
    torch.cuda.set_device(0) # Set to your desired GPU number
    print('using gpu')

home = os.path.expanduser("~")

def create_yolo_folders(path):
    """
    Creates the folder structure needed by the YOLO format.
    """

    os.mkdir(path)
    path_images = os.path.join(path, 'images')
    path_labels = os.path.join(path, 'labels')

    # Create folder structure.
    os.mkdir(path_images)
    os.mkdir(path_labels)
    os.mkdir(os.path.join(path_images, 'train'))
    os.mkdir(os.path.join(path_images, 'val'))
    os.mkdir(os.path.join(path_images, 'test'))
    os.mkdir(os.path.join(path_labels, 'train'))
    os.mkdir(os.path.join(path_labels, 'val'))
    os.mkdir(os.path.join(path_labels, 'test'))

    print(f"Created {path} directory and subdirectories.")

def convert_format_perception_to_yolo(src_folder='./solo/',
                                      dst_folder='./Converted_data/',
                                      copy_images=False,
                                      train_cutoff=70,
                                      val_cutoff=90,
                                      minVisibility=0.2):
    """
    Converts the synthetic data from the SOLO format output by the Perception package to
    the format used by YOLO. Also splits the data into a train, validation, and test set
    according to the values of dataset_size, train_cutoff, and val_cutoff.
    """

    # Create destination folders in the yolo format.
    try:
        create_yolo_folders(dst_folder)
    except OSError as error:
        print(f'Folder {dst_folder} already exists.')

    folder = 'train'
    
    # Loop over all the files.
    for (root, dirs,files) in (os.walk(src_folder)):
        dirs.sort(key=lambda directory: int(directory.split('.')[1]))

        image_nr = root.split('.')[-1]

        print(root)
        # Ignore everything except the sequence.X directories.
        if 'sequence' not in root:
            continue
        if root.endswith('sequence.0'):
            continue

        # Choose the correct folder name.
        if int(image_nr) >= train_cutoff:
            folder = 'val'
        if int(image_nr) >= val_cutoff:
            folder = 'test'

        

        # Get the images width and height. Copy the image to the YOLO images directory.
        image_path = os.path.join(root, 'step0.camera.png')
        image = Image.open(image_path, 'r')
        image_width, image_height = image.size
        image_cpy_path = os.path.join(dst_folder, 'images', folder, f'{image_nr}.png')

        if copy_images:
            image.save(image_cpy_path)
            image.close()
        else:
            image.close()
            shutil.move(image_path, image_cpy_path)

        # Read the bounding box data from the json files.
        label_path = os.path.join(root, 'step0.frame_data.json')
        with open(label_path, 'r') as f:
            example_data = json.load(f)

            

            bbox_string = ''

            try:
                values = example_data['captures'][0]['annotations'][0]['values']
                for i, bbox in enumerate(values):
                    # Calculate the center of the bounding boxes.
                    x_center = bbox['origin'][0] + 0.5 * bbox['dimension'][0]
                    y_center = bbox['origin'][1] + 0.5 * bbox['dimension'][1]

                    # Normalize the xywh as described in the YOLO Ultralytics docs:
                    # https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/
                    x = x_center / image_width
                    y = y_center / image_height
                    w = bbox['dimension'][0] / image_width
                    h = bbox['dimension'][1] / image_height
                    #remove occluded bboxes
                    if example_data['metrics'][4]['values'][i]['percentVisible'] < minVisibility:
                        continue

                    yolo_bbox = f"{bbox['labelId']} {x} {y} {w} {h}\n"
                    print(yolo_bbox)

                    bbox_string += yolo_bbox
            except KeyError:
                print('No bounding boxes')

            label_file_path = os.path.join(dst_folder, 'labels', folder, f'{image_nr}.txt')
            with open(label_file_path, 'w') as label:
                label.write(bbox_string)
    

def create_yaml(yaml_name, yolo_data,labels):
    """
    This function is used to quickly create the YOLO yaml files.
    """
    if os.path.exists(yaml_name):
        os.remove(yaml_name)

    names=''
    for l in labels:
        names+=f'    {l[0]}: {l[1]}\n'

    yaml_contents = f"""path: {yolo_data} # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test: images/test # test images (optional)

names:
{names}"""

    with open(yaml_name, 'w') as yaml_file:
        yaml_file.write(yaml_contents)

    print(f'Yaml file {yaml_name} created.')


def include_bg_photos(yolo_data, 
                      background_photos=r'./Background_photos', 
                      desired_fraction = 0.05):
    """
    Includes a fraction of unlabeled background images in the dataset 
    """
    
    desired_images = (desired_fraction/(1-desired_fraction))*(len(os.listdir(f'{yolo_data}\\images\\train'))+len(os.listdir(f'{yolo_data}\\images\\val')))
    photos = os.listdir(background_photos)
    random.shuffle(photos)
    n = min(desired_images,len(photos))
    print(f'Desired amount of bg-images={desired_images}, using {n} images.')
    
    for i,(files) in enumerate(photos):
        if i < round(0.8*n):
            shutil.copy(background_photos+f'\\{files}',f'{yolo_data}\\images\\train')
        elif i < n:
            shutil.copy(background_photos+f'\\{files}',f'{yolo_data}\\images\\val')
        else:
            print(f'{i} bg photos included.')
            break

        
def get_labels(solo_folder):
    """
    Gets the labels and corresponding id from the perception data.
    """
  
    with open(solo_folder+r'\sequence.2\step0.frame_data.json', 'r') as f:
        json_data = json.load(f)
    labels = []
    
    for IDs in json_data['metrics'][3]['values']:
        id = IDs['labelId']
        label_name = IDs['labelName']
        labels.append((id,label_name))
    print(labels)
    return labels

In [ ]:
#Data convertion
#test
#choose dataset locations
solo_folder=r'./solo'
dataset_name='cone_1000_newRand_2'
dst=f'{home}/datasets/{dataset_name}'
background_photos=r'./Background_photos'

#prepares the dataset for yolov8 training
if not os.path.exists(home + '/datasets'):
    os.mkdir(home + '/datasets')
    
labels = get_labels(solo_folder)
with open(solo_folder+'\metadata.json', 'r') as f:
        metadata = json.load(f)
dataset_size=metadata['totalFrames']
print(f'Dataset size = {dataset_size}')

convert_format_perception_to_yolo(src_folder=solo_folder,
                                  dst_folder=dst,
                                  copy_images=False, #Copy if original solo data are to be used again
                                  train_cutoff=round(dataset_size*0.8),
                                  val_cutoff=dataset_size,
                                  minVisibility=0.2)

create_yaml(f'{dst}/{dataset_name}.yaml',
             dst,labels)

include_bg_photos(dst,
                  background_photos=background_photos, 
                  desired_fraction = 0.05)


In [ ]:
# Load a model
model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)

#choose dataset
dataset_name='cone_1000_newRand_2'
dst=f'{home}/datasets/{dataset_name}'


# Train the model
results = model.train(data=f'{dst}/{dataset_name}.yaml', 
                      epochs=35, imgsz=1280, name=dataset_name, batch=-1, cache=True)

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO(r's_ubuntu_640.pt')  # load a custom trained model

# Export the model
model.export(format='onnx')